# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,12.77,73,25,4.19,SH,1698061544
1,1,angaur state,6.9060,134.1300,28.89,73,56,1.87,PW,1698061545
2,2,hermanus,-34.4187,19.2345,19.16,71,40,10.92,ZA,1698061545
3,3,stanley,54.8680,-1.6985,9.87,93,40,1.03,GB,1698061546
4,4,birjand,32.8663,59.2211,26.26,13,40,3.09,IR,1698061546


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    #hover_cols = ["Address", "Bank Count"]
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
#conditionals as variables:
max_temp = (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 25)
wind_speed = (city_data_df['Wind Speed'] < 4.5)
cloudiness = (city_data_df['Cloudiness'] == 0)

# Applying conditionals to DF
perfect_weather = pd.DataFrame(city_data_df[max_temp & wind_speed & cloudiness])

# Drop any rows with null values
perfect_weather.dropna

# Display sample data
perfect_weather.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,hawaiian paradise park,19.5933,-154.9731,23.35,95,0,2.06,US,1698061584
87,87,tomesti,47.1167,27.6833,21.74,49,0,3.60,RO,1698061598
182,182,mount isa,-20.7333,139.5000,24.87,23,0,1.54,AU,1698061657
261,261,kaka,37.3482,59.6143,23.24,35,0,3.08,TM,1698061707
383,383,kailua-kona,19.6406,-155.9956,24.57,67,0,3.09,US,1698061788
426,426,wuhan,30.5833,114.2667,23.05,44,0,3.18,CN,1698061354
548,548,pachino,36.7186,15.0907,23.74,66,0,4.02,IT,1698061916


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
64,hawaiian paradise park,US,19.5933,-154.9731,95,
87,tomesti,RO,47.1167,27.6833,49,
182,mount isa,AU,-20.7333,139.5000,23,
261,kaka,TM,37.3482,59.6143,35,
383,kailua-kona,US,19.6406,-155.9956,67,
426,wuhan,CN,30.5833,114.2667,44,
548,pachino,IT,36.7186,15.0907,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [71]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
tomesti - nearest hotel: Călina
mount isa - nearest hotel: Ibis Styles
kaka - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
wuhan - nearest hotel: 武展酒店
pachino - nearest hotel: Hotel Celeste


,City,Country,Lat,Lng,Humidity,Hotel Name
64,hawaiian paradise park,US,19.5933,-154.9731,95,No hotel found
87,tomesti,RO,47.1167,27.6833,49,Călina
182,mount isa,AU,-20.7333,139.5000,23,Ibis Styles
261,kaka,TM,37.3482,59.6143,35,No hotel found
383,kailua-kona,US,19.6406,-155.9956,67,Kona Seaside Hotel
426,wuhan,CN,30.5833,114.2667,44,武展酒店
548,pachino,IT,36.7186,15.0907,66,Hotel Celeste


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [73]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
    )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)